In [2]:
import numpy as np
import pandas as pd
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV


data = pd.read_csv('data/SN_ad.csv')
data = data.drop(['User ID'], axis=1)

dummies_gender = pd.get_dummies(data['Gender']).rename(columns=lambda x: 'Gender_' + str(x))

# userid 제거 범주화 -> 나이 범위로 나눠서 범주화

bins = [10,20,30,40,50,60]
labels = [1,2,3,4,5]

data['Age_label'] = pd.cut(data['Age'], bins=bins, labels=labels)
dummies_age = pd.get_dummies(data['Age_label']).rename(columns=lambda x: 'Age_' + str(x))

X = data.drop(['Age', 'Age_label', 'Gender'], axis=1)

Set_data = pd.concat([X, dummies_age, dummies_gender], axis=1)
Set_data

y = Set_data['Purchased']
y = y.astype('category')

X = Set_data.drop(['Purchased'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

def svc_param_selection (X,y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search

best_model = svc_param_selection(X_train, y_train, 10);

clf = best_model.best_estimator_.fit(X_train, y_train)

clf.predict(X_test)

confusion_matrix(clf.predict(X_test), y_test)